In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import numpy as np

from fnet import DCNN, Reshape
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dtype = torch.float

In [2]:
test_input = np.random.uniform(10,400,[10,20])
test_input = torch.tensor(test_input,device=device,dtype=dtype)


In [3]:
fnet = DCNN(20,121).to(device)

In [4]:
fnet(test_input)

tensor([[0.4601, 0.4758, 0.7681,  ..., 0.5269, 0.3000, 0.3461],
        [0.5000, 0.4919, 0.7701,  ..., 0.5343, 0.3173, 0.3273],
        [0.4255, 0.4819, 0.7557,  ..., 0.5000, 0.3174, 0.3385],
        ...,
        [0.4224, 0.5000, 0.7613,  ..., 0.5101, 0.5000, 0.3537],
        [0.4378, 0.5060, 0.7799,  ..., 0.5396, 0.3159, 0.3394],
        [0.4369, 0.5000, 0.7578,  ..., 0.5761, 0.3309, 0.3627]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)

In [ ]:
conv_layer = nn.Conv1d(1, 1, kernel_size=101,padding=50)
conv_test = np.random.rand(100,1,800)
conv_test = torch.tensor(conv_test,dtype=dtype)
batch_norm = nn.BatchNorm1d(1)
rs = Reshape(-1,1,800)

In [ ]:
rs(conv_test).shape

In [ ]:
conv_layer(rs(conv_test)).shape

In [ ]:
batch_norm(conv_test)

In [ ]:
conv_test